In [1]:
import pickle
import torch
import random
import torch.nn as nn
import numpy as np
from collections import namedtuple
import matplotlib
import matplotlib.pyplot as plt
import torch.optim as optim
import torch.nn.functional as F

In [2]:
# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# initialize numpy random seed
np.random.seed(57)
random_gen = np.random.default_rng()

In [3]:
'''
    Takes a state or batch of states and converts
    them into a pytorch tensor format. Shapes should
    be (batch_size x 1 x 4 x 4) or (1 x 1 x 4 x 4)
'''
def process_state(state):
    if state is not None:
        state = torch.tensor(state, dtype=torch.float)
        state = state.unsqueeze(0).unsqueeze(0).to(device)
    return state
  
def process_action(action):
    if action == 'Left':
        out = 0
    elif action == 'Right':
        out = 1
    elif action == 'Up':
        out = 2
    elif action == 'Down':
        out = 3
    
    return torch.tensor([[out]], device=device, dtype=torch.long)

def process_reward(reward):
    return torch.tensor([reward], device=device, dtype=torch.long)


In [4]:
# Set up transition and ReplayMemory classes
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))


class ReplayMemory(object):
    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    def push(self, *args):
        """Saves a transition."""
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.position] = Transition(*args)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        # Gets a minibatch of tuples
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [5]:
# Load dataset and shuffle
raw_data = pickle.load(open('dataset.p', 'rb'))

dataset = []
for episode in raw_data:
    for exp in raw_data[episode]['experiences']:
        dataset.append(exp)

random_gen.shuffle(dataset)
print('Dataset has {} experiences'.format(len(dataset)))

Dataset has 60381 experiences


In [6]:
# Push entire dataset into ReplayMemory
memory = ReplayMemory(10000)

for exp in dataset[:10000]:
    state, action, nextstate, reward = exp
    state = process_state(state)
    nextstate = process_state(state)
    action = process_action(action)
    reward = process_reward(reward)
    memory.push(state, action, nextstate, reward)

# Test sample method
print(memory.sample(2))

<ipython-input-3-e93d019ebf59>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  state = torch.tensor(state, dtype=torch.float)


[Transition(state=tensor([[[[ 32.,  32.,   2.,   0.],
          [  4., 128.,   4.,   2.],
          [  8.,   2.,   8.,   0.],
          [  2.,   4.,   2.,   2.]]]]), action=tensor([[2]]), next_state=tensor([[[[[[ 32.,  32.,   2.,   0.],
            [  4., 128.,   4.,   2.],
            [  8.,   2.,   8.,   0.],
            [  2.,   4.,   2.,   2.]]]]]]), reward=tensor([4])), Transition(state=tensor([[[[ 0.,  0.,  2.,  0.],
          [ 2.,  0., 16.,  0.],
          [ 2.,  4.,  4.,  8.],
          [64., 16.,  2.,  8.]]]]), action=tensor([[1]]), next_state=tensor([[[[[[ 0.,  0.,  2.,  0.],
            [ 2.,  0., 16.,  0.],
            [ 2.,  4.,  4.,  8.],
            [64., 16.,  2.,  8.]]]]]]), reward=tensor([8]))]


In [7]:
transitions = memory.sample(16)
batch = Transition(*zip(*transitions))
non_final_next_states = torch.cat([s.squeeze(1).squeeze(1) for s in batch.next_state
                                                if s is not None])
print(non_final_next_states.shape)

torch.Size([16, 1, 4, 4])


In [8]:
# Define network

class DQN(nn.Module):
    def __init__(self, h, w, outputs):
        super(DQN, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, kernel_size=4, padding=1, stride=1)
        self.bn1 = nn.BatchNorm2d(8)
        self.conv2 = nn.Conv2d(8, 16, kernel_size=2, padding=1, stride=1)
        self.bn2 = nn.BatchNorm2d(16)

        # Number of Linear input connections depends on output of conv2d layers
        # and therefore the input image size, so compute it.
        def conv2d_size_out(size, kernel_size, padding=1, stride = 1):
            return (size  +2*padding - (kernel_size - 1) - 1) // stride  + 1
        
        convw = conv2d_size_out(conv2d_size_out(w, 4), 2)
        convh = conv2d_size_out(conv2d_size_out(h, 4), 2)
        linear_input_size = convw * convh * 16
        self.head = nn.Linear(linear_input_size, outputs)

    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        return self.head(x.view(x.size(0), -1))

In [9]:
BATCH_SIZE = 128
GAMMA = 0.999
TARGET_UPDATE = 20

board_width = 4
board_height = 4
n_actions = 4

policy_net = DQN(board_height, board_width, n_actions).to(device)
target_net = DQN(board_height, board_width, n_actions).to(device)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

optimizer = optim.RMSprop(policy_net.parameters())

steps_done = 0


def select_action(state):
    global steps_done
    steps_done += 1
    with torch.no_grad():
        # t.max(1) will return largest column value of each row.
        # second column on max result is index of where max element was
        # found, so we pick action with the larger expected reward.
        return policy_net(state).max(1)[1].view(1, 1)

episode_durations = []

def plot_durations():
    plt.figure(2)
    plt.clf()
    durations_t = torch.tensor(episode_durations, dtype=torch.float)
    plt.title('Training...')
    plt.xlabel('Episode')
    plt.ylabel('Duration')
    plt.plot(durations_t.numpy())
    # Take 100 episode averages and plot them too
    if len(durations_t) >= 100:
        means = durations_t.unfold(0, 100, 1).mean(1).view(-1)
        means = torch.cat((torch.zeros(99), means))
        plt.plot(means.numpy())

    plt.pause(0.001)  # pause a bit so that plots are updated
    if is_ipython:
        display.clear_output(wait=True)
        display.display(plt.gcf())

In [10]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s.squeeze(1).squeeze(1) for s in batch.next_state
                                                if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    state_action_values = policy_net(state_batch).gather(1, action_batch)

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1)[0].
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    next_state_values[non_final_mask] = target_net(non_final_next_states).max(1)[0].detach()
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    loss = F.smooth_l1_loss(state_action_values, expected_state_action_values.unsqueeze(1))
    print(loss)
    
    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    for param in policy_net.parameters():
        param.grad.data.clamp_(-1, 1)
    optimizer.step()

In [11]:
train_iterations = 78
for i_train in range(train_iterations):
    # Update policy network
    optimize_model()
    
    # Update the target network, copying all weights and biases in DQN
    if i_train % TARGET_UPDATE == 0:
        print('Updating Target Network')
        target_net.load_state_dict(policy_net.state_dict())

print('Complete')

tensor(12.3720, grad_fn=<SmoothL1LossBackward>)
Updating Target Network
tensor(15.2468, grad_fn=<SmoothL1LossBackward>)
tensor(8.8376, grad_fn=<SmoothL1LossBackward>)
tensor(12.4419, grad_fn=<SmoothL1LossBackward>)
tensor(7.7604, grad_fn=<SmoothL1LossBackward>)
tensor(7.5426, grad_fn=<SmoothL1LossBackward>)
tensor(7.6105, grad_fn=<SmoothL1LossBackward>)
tensor(7.6574, grad_fn=<SmoothL1LossBackward>)
tensor(8.5910, grad_fn=<SmoothL1LossBackward>)
tensor(5.2145, grad_fn=<SmoothL1LossBackward>)
tensor(9.2784, grad_fn=<SmoothL1LossBackward>)
tensor(6.2149, grad_fn=<SmoothL1LossBackward>)
tensor(7.0058, grad_fn=<SmoothL1LossBackward>)
tensor(9.3782, grad_fn=<SmoothL1LossBackward>)
tensor(7.0541, grad_fn=<SmoothL1LossBackward>)
tensor(8.4104, grad_fn=<SmoothL1LossBackward>)
tensor(8.4153, grad_fn=<SmoothL1LossBackward>)
tensor(12.9083, grad_fn=<SmoothL1LossBackward>)
tensor(7.7156, grad_fn=<SmoothL1LossBackward>)
tensor(8.4570, grad_fn=<SmoothL1LossBackward>)
tensor(6.7435, grad_fn=<SmoothL1

In [12]:
torch.save(policy_net.state_dict(), 'trained_model.pt')

In [13]:
inf_model = DQN(board_height, board_width, n_actions).to(device)
inf_model.load_state_dict(torch.load('trained_model.pt'))
inf_model.eval()

DQN(
  (conv1): Conv2d(1, 8, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(8, 16, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (head): Linear(in_features=256, out_features=4, bias=True)
)

In [14]:
cur_state = memory.sample(1)[0]
model_out = inf_model(cur_state.state)
print(model_out)
print(np.argsort(model_out.data.numpy()[0]))
# print(model_out
# move = model_out.max(1)[1].numpy()[0]

tensor([[18.9898, 18.7269, 18.1037, 19.7032]], grad_fn=<AddmmBackward>)
[2 1 0 3]


In [15]:
ranked_actions = np.argsort(model_out.data.numpy()[0])
chosen_action = ranked_actions[0]
# chosen_move = move_ops[chosen_action]

while True:
    ranked_actions = ranked_actions[ranked_actions != chosen_action]
    print(ranked_actions)
    if len(ranked_actions) == 0:
        continue_playing = "Invalid";
        break;
    chosen_action = ranked_actions[0]
#     chosen_move = ranked_actions[0]

[1 0 3]
[0 3]
[3]
[]
